In [1]:
file = open("prototype2.circ", "r")
contents = file.readlines()
file.close()

In [2]:
from parse import *
class Wires:
    def __init__(self, offset_x, offset_y, step_x, step_y):
        self.offset_x = offset_x
        self.offset_y = offset_y
        self.step_x = step_x
        self.step_y = step_y
    
    def parse_from_lines(self, lines):
        coords_names = []
        for line in lines:
            res = parse('    <wire from="({},{})" to="({},{})"/>\n', line)
            if res:
                coords_names.append(res.fixed)
        self.wires = [(int(a), int(b), int(c), int(d)) for a, b, c, d in coords_names]
    
    def offset(self, offset_x, offset_y):
        self.offset_x += offset_x
        self.offset_y += offset_y

    def to_list(self, i, j):
        get_x = lambda x: x + self.offset_x + i*self.step_x
        get_y = lambda y: y + self.offset_y + j*self.step_y
        return [f'      <wire from="({get_x(x[0])},{get_y(x[1])})" to="({get_x(x[2])},{get_y(x[3])})"/>\n' for x in self.wires]

In [3]:
class Tunnels:
    def __init__(self, offset_x, offset_y, step_x, step_y):
        self.offset_x = offset_x
        self.offset_y = offset_y
        self.step_x = step_x
        self.step_y = step_y
    
    def parse_from_lines(self, lines):
        coords_names = []
        orientations = []
        in_tunnel = False
        for line in lines:
            res = parse('    <comp lib="0" loc="({},{})" name="Tunnel">\n', line)
            if res:
                if len(orientations) < len(coords_names):
                    orientations.append("west")
                coords_names.append(res.fixed)
                in_tunnel = True
            if line == '    </comp>':
                in_tunnel = False
                
            orientation = parse('      <a name="facing" val="{}"/>\n', line)
            if in_tunnel and orientation:
                orientations.append(orientation.fixed[0])
        if len(orientations) < len(coords_names):
            orientations.append("west")
                
        self.tunnels = sorted([(int(a), int(b), o) for (a, b), o in zip(coords_names, orientations)], key=lambda x: x[1])
    
    def offset(self, offset_x, offset_y):
        self.offset_x += offset_x
        self.offset_y += offset_y

    def to_list(self, i, j, name):
        get_x = lambda x: x + self.offset_x + i*self.step_x
        get_y = lambda y: y + self.offset_y + j*self.step_y
        return [f"""<comp lib="0" loc="({get_x(x)},{get_y(y)})" name="Tunnel">
      <a name="facing" val="{orient}"/>
      <a name="label" val="{name(orient, index)}"/>
      <a name="labelfont" val="SansSerif plain 9"/>
    </comp>\n""" for index, (x, y, orient) in enumerate(self.tunnels)]

In [4]:
start_x = 160
start_y = 440
step_x = 380
step_y = 850

pins_wires = Wires(0, 0, step_x, step_y)
pins_wires.parse_from_lines(contents[6306:6399])
pins_tunnels = Tunnels(0, 0, step_x, step_y)
pins_tunnels.parse_from_lines(contents[6306:])

In [5]:
circuit = []
for i in range(32):
    circuit += f"""    <comp lib="0" loc="({start_x+i*step_x},{start_y})" name="Splitter">
      <a name="fanout" val="32"/>
      <a name="incoming" val="32"/>
      <a name="appear" val="center"/>
    </comp>\n"""
    circuit += f"""    <comp lib="0" loc="({start_x+i*step_x},{start_y})" name="Pin">
      <a name="width" val="32"/>
      <a name="tristate" val="false"/>
      <a name="label" val="row{i}"/>
    </comp>\n"""
    circuit += pins_wires.to_list(i, 0)
    circuit += pins_tunnels.to_list(i, 0, lambda _, index: f"cell{i}_{index}")

In [6]:
for i in range(32):
    circuit += f"""    <comp lib="0" loc="({start_x+i*step_x},{start_y+step_y})" name="Splitter">
      <a name="fanout" val="32"/>
      <a name="incoming" val="32"/>
      <a name="appear" val="center"/>
    </comp>\n"""
    circuit += f"""    <comp lib="0" loc="({start_x+i*step_x},{start_y+step_y})" name="Pin">
      <a name="output" val="true"/>
      <a name="width" val="32"/>
      <a name="tristate" val="false"/>
      <a name="label" val="orow{i}"/>
    </comp>\n"""
    circuit += pins_wires.to_list(i, 1)
    circuit += pins_tunnels.to_list(i, 1, lambda _, index: f"ocell{i}_{index}")


In [7]:
import generate
generate.rewrite_circuit(circuit)

In [8]:
file = open("prototype_corner.circ", "r")
corner_contents = file.readlines()
file.close()

In [9]:
offset_x = 0
offset_y = 1400
step_x = 150
step_y = 150

corner_wires = Wires(offset_x, offset_y, step_x, step_y)
corner_wires.parse_from_lines(corner_contents[6306:])
corner_tunnels = Tunnels(offset_x, offset_y, step_x, step_y)
corner_tunnels.parse_from_lines(corner_contents[6306:])

In [10]:
from itertools import product
neighbors = lambda x, y: [(x+i, y+j) for i, j in product(range(-1, 2), range(-1, 2)) if abs(i)+abs(j) < 2]
valid_coord = lambda coords: coords[0] >= 0 and coords[1] >= 0 and coords[0] <= 31 and coords[1] <= 31
for i in [0, 31]:
    for j in [0, 31]:
        cell_neighbors = list(map(lambda x: f"{x[0]}_{x[1]}", filter(valid_coord, neighbors(j, i))))
        circuit += corner_wires.to_list(i, j)
        circuit += corner_tunnels.to_list(i, j, lambda orient, index: f"ocell{j}_{i}" if orient == "west" else f"cell{cell_neighbors.pop()}")
        circuit += f"""     <comp lib="1" loc="({220+offset_x+j*step_x},{250+offset_y+i*step_y})" name="OR Gate">
      <a name="size" val="30"/>
      <a name="inputs" val="3"/>
    </comp>\n"""


In [11]:
import generate
generate.rewrite_circuit(circuit)

In [12]:
file = open("prototype_edge.circ", "r")
edge_contents = file.readlines()
file.close()

In [13]:
edge_wires = Wires(offset_x, offset_y, step_x, step_y)
edge_wires.parse_from_lines(edge_contents[6306:])
edge_tunnels = Tunnels(offset_x, offset_y, step_x, step_y)
edge_tunnels.parse_from_lines(edge_contents[6306:])

In [14]:
for i in [0, 31]:
    for j in range(1, 31):
        cell_neighbors = list(map(lambda x: f"{x[0]}_{x[1]}", filter(valid_coord, neighbors(j, i))))
        circuit += edge_wires.to_list(i, j)
        circuit += edge_tunnels.to_list(i, j, lambda orient, index: f"ocell{j}_{i}" if orient == "west" else f"cell{cell_neighbors.pop()}")
        circuit += f"""     <comp lib="1" loc="({220+offset_x+j*step_x},{250+offset_y+i*step_y})" name="OR Gate">
      <a name="size" val="30"/>
      <a name="inputs" val="4"/>
    </comp>\n"""
for i in range(1, 31):
    for j in [0, 31]:
        cell_neighbors = list(map(lambda x: f"{x[0]}_{x[1]}", filter(valid_coord, neighbors(j, i))))
        circuit += edge_wires.to_list(i, j)
        circuit += edge_tunnels.to_list(i, j, lambda orient, index: f"ocell{j}_{i}" if orient == "west" else f"cell{cell_neighbors.pop()}")
        circuit += f"""     <comp lib="1" loc="({220+offset_x+j*step_x},{250+offset_y+i*step_y})" name="OR Gate">
      <a name="size" val="30"/>
      <a name="inputs" val="4"/>
    </comp>\n"""

In [15]:
import generate
generate.rewrite_circuit(circuit)

In [16]:
file = open("prototype_center.circ", "r")
center_contents = file.readlines()
file.close()

In [17]:
center_wires = Wires(offset_x, offset_y, step_x, step_y)
center_wires.parse_from_lines(center_contents[6306:])
center_tunnels = Tunnels(offset_x, offset_y, step_x, step_y)
center_tunnels.parse_from_lines(center_contents[6306:])

In [18]:
for i in range(1, 31):
    for j in range(1, 31):
        cell_neighbors = list(map(lambda x: f"{x[0]}_{x[1]}", filter(valid_coord, neighbors(j, i))))
        circuit += center_wires.to_list(i, j)
        circuit += center_tunnels.to_list(i, j, lambda orient, index: f"ocell{j}_{i}" if orient == "west" else f"cell{cell_neighbors.pop()}")
        circuit += f"""     <comp lib="1" loc="({220+offset_x+j*step_x},{250+offset_y+i*step_y})" name="OR Gate">
      <a name="size" val="30"/>
      <a name="inputs" val="5"/>
    </comp>\n"""

In [19]:
import generate
generate.rewrite_circuit(circuit)